[example-logistic-regression](https://github.com/torch/demos/blob/master/logistic-regression/example-logistic-regression.lua)


In [47]:
require 'nn'
require 'optim'

In [48]:
local matio = require 'matio'
data = matio.load('ex3data1.mat')

In [49]:
dataset_inputs = data.X

In [50]:
dataset_outputs = data.y

In [51]:
numOutput = torch.max(dataset_outputs) - torch.min(dataset_outputs) + 1

In [52]:
linLayer = nn.Linear(400,10)

In [53]:
model = nn.Sequential()
model:add(linLayer)
model:add(nn.Sigmoid())

In [54]:
model

nn.Sequential {
  [input -> (1) -> (2) -> output]
  (1): nn.Linear(400 -> 10)
  (2): nn.Sigmoid
}
{
  gradInput : DoubleTensor - empty
  modules : 
    {
      1 : 
        nn.Linear(400 -> 10)
        {
          gradBias : DoubleTensor - size: 10
          weight : DoubleTensor - size: 10x400
          bias : DoubleTensor - size: 10
          gradInput : DoubleTensor - empty
          gradWeight : DoubleTensor - size: 10x400
          output : DoubleTensor - empty
        }
      2 : 
        nn.Sigmoid
        {
          gradInput : DoubleTensor - empty
          output : DoubleTensor - empty
        }
    }
  output : DoubleTensor - empty
}


In [55]:
criterion = nn.ClassNLLCriterion()

In [56]:
x, dl_dx = model:getParameters()

In [57]:
feval = function()
    _nidx_ = (_nidx_ or 0) + 1
    if _nidx_ > (#dataset_inputs)[1] then _nidx_ = 1 end
    
    local inputs = dataset_inputs[_nidx_]
    local target = dataset_outputs[_nidx_]
    
    dl_dx:zero()
    
    local loss_x = criterion:forward(model:forward(inputs), target)
    model:backward(inputs, criterion:backward(model.output, target))
        
    return loss_x, dl_dx
end

In [58]:
sgd_params = {
    learningRate = 1e-2,
    learningRateDecay = 1e-3,
    weightDecay = 0,
    momentum = 0
}

In [59]:
epochs = 1e2

In [60]:
for i = 1, epochs do
    current_loss = 0
    for i = 1, (#dataset_inputs)[1] do
        _, fs = optim.sgd(feval, x, sgd_params)
        current_loss = current_loss + fs[1]
    end
    
    current_loss = current_loss / (#dataset_inputs)[1]
    if i%10 == 1 then
        print('epoch = ' .. i .. ' of ' .. epochs .. ' current loss = ' .. current_loss)
    end
end

epoch = 1 of 100 current loss = -0.86834777668181	


epoch = 11 of 100 current loss = -0.98178234765091	


epoch = 21 of 100 current loss = -0.98452670577965	


epoch = 31 of 100 current loss = -0.98578519432083	


epoch = 41 of 100 current loss = -0.98656001749766	


epoch = 51 of 100 current loss = -0.98710421963983	


epoch = 61 of 100 current loss = -0.98751640282183	


epoch = 71 of 100 current loss = -0.98784427708077	


epoch = 81 of 100 current loss = -0.98811421918889	


epoch = 91 of 100 current loss = -0.98834220344355	


In [61]:
function maxIndex(a)
    local idx = 0
    local mx = -1e5
    for i = 1,(#a)[1] do
        if mx < a[i] then
            mx = a[i]
            idx = i
        end
    end
    return idx
end

In [62]:
acc = 0.0
for i = 1, (#dataset_inputs)[1] do
    if (maxIndex(model:forward(dataset_inputs[i])) == dataset_outputs[i][1]) then
        acc = acc + 1
    end
end
print('accuracy: ' .. acc/(#dataset_inputs)[1])

accuracy: 0.6912	
